In [1]:
# Library Imports
import pandas as pd
from sqlalchemy import create_engine 
from Password import password

In [2]:
# Links to respective CSV Files
hospital_locations = "Resources/hospital_locations.csv"
hospital_general_info = "Resources\Hospital_General_Information.csv"


# Convert CSVs to DataFrames
hospital_locations_df = pd.read_csv(hospital_locations)
hospital_general_info_df = pd.read_csv(hospital_general_info)

In [3]:
#Confirm that Provider ID is unique value which is unrepeating throughout the dataset. Suitable for Primary Key
hospital_general_info_df["Provider ID"].value_counts()

141311    1
453304    1
230077    1
60096     1
230081    1
         ..
521313    1
171373    1
370030    1
30064     1
141313    1
Name: Provider ID, Length: 4818, dtype: int64

In [4]:
# The purpose of this DF is to provide the address and contact information for various hospitals. Unneed Columns are stripped.
# The columns listed below were used in the General Hospital DataFrame

hospital_general_info_df = hospital_general_info_df[[
    "Provider ID",
    "Hospital Name",
    "Address",
    "City",
    "State",
    "ZIP Code",
    "County Name",
    "Phone Number",
    "Hospital Ownership",
    "Hospital Type",
    "Emergency Services",
    "Hospital overall rating"
]]

In [5]:
# Correct Object Datatypes into Strings, or other appropriate datatype. 

hospital_general_info_df.astype({
    "Hospital Name":"string",
    "Address":"string",
    "City":"string",
    "State":"string",
    "County Name":"string",
    "Hospital Ownership":"string",
    "Hospital Type":"string"
})

Provider ID                       Hospital Name  \
0           10005       MARSHALL MEDICAL CENTER SOUTH   
1           10032                    WEDOWEE HOSPITAL   
2           10131            CRESTWOOD MEDICAL CENTER   
3           20001    PROVIDENCE ALASKA MEDICAL CENTER   
4           20018  YUKON KUSKOKWIM DELTA REG HOSPITAL   
...           ...                                 ...   
4813       501333  KITTITAS VALLEY COMMUNITY HOSPITAL   
4814       450827         KELL WEST REGIONAL HOSPITAL   
4815       501310          NEWPORT COMMUNITY HOSPITAL   
4816       670107     BAYLOR EMERGENCY MEDICAL CENTER   
4817       450213            UNIVERSITY HEALTH SYSTEM   

                            Address           City State  ZIP Code  \
0        2505 U S HIGHWAY 431 NORTH           BOAZ    AL     35957   
1             209 NORTH MAIN STREET        WEDOWEE    AL     36278   
2                ONE HOSPITAL DR SE     HUNTSVILLE    AL     35801   
3                        BOX 196604      ANCHORAGE    AK     99508   
4                        PO BOX 287         BETHEL    AK     99559   
...                             ...            ...   ...       ...   
4813             603 SOUTH CHESTNUT     ELLENSBURG    WA     98926   
4814       5402 KELL WEST BOULEVARD  WICHITA FALLS    TX     76310   
4815           714 WEST PINE STREET        NEWPORT    WA     99156   
4816  12500 SOUTH FREEWAY SUITE 100       BURLESON    TX     76028   
4817                4502 MEDICAL DR    SAN ANTONIO    TX     78229   

       County Name  Phone Number                           Hospital Ownership  \
0         MARSHALL    2565938310  Government - Hospital District or Authority   
1         RANDOLPH    2563572111  Government - Hospital District or Authority   
2          MADISON    2568823100                                  Proprietary   
3        ANCHORAGE    9072123675                Voluntary non-profit - Church   
4           BETHEL    9075436300                                       Tribal   
...            ...           ...                                          ...   
4813      KITTITAS    5099629841  Government - Hospital District or Authority   
4814       WICHITA    9406925888                                  Proprietary   
4815  PEND OREILLE    5094472441  Government - Hospital District or Authority   
4816       JOHNSON    2142946250                                  Proprietary   
4817         BEXAR    2103584000  Government - Hospital District or Authority   

                  Hospital Type  Emergency Services  Hospital overall rating  
0          Acute Care Hospitals                True                      3.0  
1          Acute Care Hospitals                True                      4.0  
2          Acute Care Hospitals                True                      3.0  
3          Acute Care Hospitals                True                      3.0  
4          Acute Care Hospitals                True                      3.0  
...                         ...                 ...                      ...  
4813  Critical Access Hospitals                True                      3.0  
4814       Acute Care Hospitals                True                      4.0  
4815  Critical Access Hospitals                True                      NaN  
4816       Acute Care Hospitals                True                      NaN  
4817       Acute Care Hospitals                True                      2.0  

[4818 rows x 12 columns]

In [6]:
new_hospital_general_info_df = hospital_general_info_df.rename(columns={
    "Provider ID":"provider_id",
    "Hospital Name":"hospital_name",
    "Address":"address",
    "ZIP Code":"zip_code",
    "City":"city",
    "State":"state",
    "County Name":"county_name",
    "Phone Number":"phone_number",
    "Hospital Ownership":"hospital_ownership",
    "Hospital Type":"hospital_type",
    "Emergency Services":"emergency_services",
    "Hospital overall rating":"hospital_overall_rating"
})


In [7]:
# .info() used to identify columns which have missing data. county_name, hospital_overall_rating both have null values. 
new_hospital_general_info_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4818 entries, 0 to 4817
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   provider_id              4818 non-null   int64  
 1   hospital_name            4818 non-null   object 
 2   address                  4818 non-null   object 
 3   city                     4818 non-null   object 
 4   state                    4818 non-null   object 
 5   zip_code                 4818 non-null   int64  
 6   county_name              4803 non-null   object 
 7   phone_number             4818 non-null   int64  
 8   hospital_ownership       4818 non-null   object 
 9   hospital_type            4818 non-null   object 
 10  emergency_services       4818 non-null   bool   
 11  hospital_overall_rating  3648 non-null   float64
dtypes: bool(1), float64(1), int64(3), object(7)
memory usage: 418.9+ KB


In [8]:
# Fill the null values with 'N/A'.

new_hospital_general_info_df["hospital_overall_rating"].fillna("N/A", inplace=True)
new_hospital_general_info_df["county_name"].fillna("N/A", inplace=True)

In [15]:
protocol = 'postgresql'
username = 'postgres'
password = password
host = 'localhost'
port = 5432
database_name = 'Hospital_DB'
rds_connection_string = f'{protocol}://{username}:{password}@{host}:{port}/{database_name}'
engine = create_engine(rds_connection_string)

In [16]:
engine = create_engine(rds_connection_string)

In [17]:
# Upload DataFrame to SQL Database

new_hospital_general_info_df.to_sql(name="hospital_location", con=engine, if_exists="replace", index=False)

In [18]:
# Query the Data

pd.read_sql_query('select * from hospital_location', con=engine).head()

provider_id                       hospital_name  \
0        10005       MARSHALL MEDICAL CENTER SOUTH   
1        10032                    WEDOWEE HOSPITAL   
2        10131            CRESTWOOD MEDICAL CENTER   
3        20001    PROVIDENCE ALASKA MEDICAL CENTER   
4        20018  YUKON KUSKOKWIM DELTA REG HOSPITAL   

                      address        city state  zip_code county_name  \
0  2505 U S HIGHWAY 431 NORTH        BOAZ    AL     35957    MARSHALL   
1       209 NORTH MAIN STREET     WEDOWEE    AL     36278    RANDOLPH   
2          ONE HOSPITAL DR SE  HUNTSVILLE    AL     35801     MADISON   
3                  BOX 196604   ANCHORAGE    AK     99508   ANCHORAGE   
4                  PO BOX 287      BETHEL    AK     99559      BETHEL   

   phone_number                           hospital_ownership  \
0    2565938310  Government - Hospital District or Authority   
1    2563572111  Government - Hospital District or Authority   
2    2568823100                                  Proprietary   
3    9072123675                Voluntary non-profit - Church   
4    9075436300                                       Tribal   

          hospital_type  emergency_services hospital_overall_rating  
0  Acute Care Hospitals                True                     3.0  
1  Acute Care Hospitals                True                     4.0  
2  Acute Care Hospitals                True                     3.0  
3  Acute Care Hospitals                True                     3.0  
4  Acute Care Hospitals                True                     3.0